## Build a contact network

# Download dataset

In [1]:
!wget https://lp-prod-resources.s3.amazonaws.com/628/66549/2021-06-25-19-30-14/PeopleLocations.csv

--2023-02-20 11:06:12--  https://lp-prod-resources.s3.amazonaws.com/628/66549/2021-06-25-19-30-14/PeopleLocations.csv
Resolving lp-prod-resources.s3.amazonaws.com (lp-prod-resources.s3.amazonaws.com)... 52.218.153.211, 52.92.146.33, 52.92.192.137, ...
Connecting to lp-prod-resources.s3.amazonaws.com (lp-prod-resources.s3.amazonaws.com)|52.218.153.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16055 (16K) [application/octet-stream]
Saving to: ‘PeopleLocations.csv’

PeopleLocations.csv 100%[===================>]  15.68K  72.6KB/s    in 0.2s    

2023-02-20 11:06:14 (72.6 KB/s) - ‘PeopleLocations.csv’ saved [16055/16055]



In [15]:
import numpy as np
import pandas as pd
import networkx as nx

In [16]:
# read in csv to pd.DataFrame
df = pd.read_csv("PeopleLocations.csv", sep = ";", 
                 parse_dates=['Date'], dtype={'id': np.str_, 'Lat': np.float32, 'Lon': np.float32})

In [17]:
# convert type to category
df["Covid19"] = df["Covid19"].astype("category")

In [18]:
# check the dtypes
df.dtypes

ID                 object
Lat               float32
Lon               float32
Date       datetime64[ns]
Time               object
Covid19          category
dtype: object

In [26]:
# create a graph object
G = nx.Graph()

In [41]:
G.add_nodes_from({"names":df.ID})

In [45]:
G.nodes()

NodeView(('Person1', 'Person2', 'Person3', 'Person4', 'Person5', 'Person6', 'Person7', 'Person8', 'Person9', 'Person10', 'Person11', 'Person12', 'Person13', 'Person14', 'Person15', 'Person16', 'Person17', 'Person18', 'Person19', 'Person20', 'Person21', 'Person22', 'Person23', 'Person24', 'Person25', 'Person26', 'Person27', 'Person28', 'Person29', 'Person30', 'Person31', 'Person32', 'Person33', 'Person34', 'Person35', 'Person36', 'Person37', 'Person38', 'Person39', 'Person40', 'Person41', 'Person42', 'Person43', 'Person44', 'Person45', 'Person46', 'Person47', 'Person48', 'Person49', 'Person50', 'Person51', 'Person52', 'Person53', 'Person54', 'Person55', 'Person56', 'Person57', 'Person58', 'Person59', 'Person60', 'Person61', 'Person62', 'Person63', 'Person64', 'Person65', 'Person66', 'Person67', 'Person68', 'Person69', 'Person70', 'Person71', 'Person72', 'Person73', 'Person74', 'Person75', 'Person76', 'Person77', 'Person78', 'Person79', 'Person80', 'Person81', 'Person82', 'Person83', 'Pe